In [ ]:
import azureml.core
from azureml.core import Workspace, Model

# Inference testing function definition

In [ ]:
import base64
import requests
import json 

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

def test_inf(endpoint, key=None):
    input_json = json.dumps({
        "n_cards": 5,
        "images": True,
        "hand": get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/test/IMG_4903.jpg'),
        "deck": get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/train/A_1.jpg')
    })
    headers = { 'Content-Type':'application/json' }
    if key:
        headers['Authorization'] = f'Bearer {key}' # Only difference to local is to send the auth key
    predictions = requests.post(endpoint, input_json, headers = headers)
    p = predictions.json()
    print(str(p)+'\n')
    print(f'Hand:     {p[0]}')
    print(f'Deck:     {p[1]}')
    if p[2]:
        play = p[2][0]
        print(f'Score:    {play[0]}')
        print(f'Complete: {play[1]}')
        print(f'Words:    {[c[0] for c in play[2]]}')
        print(f'Pick up:  {p[2][1]}')
        print(f'Drop:     {p[2][2]}\n')
    else:
        print('No possible play for these cards')    

# Connect to the Workspace

In [ ]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print(f'Azure ML version: {azureml.core.VERSION}, Workspace: {ws.name}')

# Create the inference environment

In [ ]:
from azureml.core.webservice import LocalWebservice, AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

In [ ]:
myenv = Environment(name="quiddler-inference-env")
myenv.inferencing_stack_version='latest'
myenv.docker.enabled = True
myenv.docker.base_image = None

# Specify docker steps as a string.
# see https://github.com/Azure/AzureML-Containers
# Needed to add libgl1-mesa-dev to resolve:
#  ImportError: libGL.so.1: cannot open shared object file: No such file or directory
dockerfile = r'''
FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
RUN apt-get install -y libgl1-mesa-dev
'''
myenv.docker.base_dockerfile = dockerfile

conda_dep = CondaDependencies(conda_dependencies_file_path='inference-env.yml')
myenv.python.conda_dependencies=conda_dep

# Configure the scoring environment
inference_config = InferenceConfig(source_directory = 'source',
                                   entry_script="score.py",
                                   environment=myenv)

In [ ]:
model = ws.models['quiddler_model']
print(f'Model: {model.name}, Version: {model.version}')

# Deploy locally

In [ ]:
deployment_config = LocalWebservice.deploy_configuration()

service_name = "quiddler-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

## Call the inference service

In [ ]:
# Reload the service to pick up changes in the source directory
service.reload(wait=True)

In [ ]:
local_endpoint = service.scoring_uri
print(local_endpoint)

In [ ]:
test_inf(local_endpoint)

In [ ]:
print(service.get_logs())

## Shut it down

In [ ]:
service.delete()
print("Service deleted.")

In [ ]:
!docker container ls

# Deploy to Azure Container Instance

In [ ]:
# ACI Service
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights=True)

service_name = "quiddler-service"

aci_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
aci_endpoint = aci_service.scoring_uri
print(aci_endpoint)

In [ ]:
test_inf(aci_endpoint)

In [ ]:
aci_service.delete()

# Deploy into AKS

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute

service_name = "quiddler-service2"
aks_target = AksCompute(ws,"jb-inf")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aks_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)
#print(aks_service.get_logs())

In [ ]:
primary, secondary = aks_service.get_keys()
aks_endpoint = aks_service.scoring_uri
print(aks_endpoint)
print(primary)

In [ ]:
test_inf(aks_endpoint, primary)

In [ ]:
aks_service.delete()